# Collaborative Filtering Recommender System based on "Deep" Autoencoders

### Purpose
To test a vanilla "deep" autoencoder model based off of collaborative filtering using Keras and Tensorflow

### Methodology
This notebook assumes that the model will receive a pre-processed dataset of user-item interactions. For simplification purposes, it uses the [small movielens dataset](https://surprise.readthedocs.io/en/stable/dataset.html)

### Author Information
Nishant Aswani (@niniack)


# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, Predictor, als, basic, user_knn
from lenskit import topn
from scipy import sparse
from lenskit.data import sparse_ratings

# Dataset
from lenskit.datasets import ML100K, ML1M

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations and debugging
import plotly.graph_objs as go
from pprintpp import pprint as pp
import logging

# Tensorflow
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

### Downloading ML100K Dataset

In [2]:
# %%!
# wget -q -O ml-100k.zip http://files.grouplens.org/datasets/movielens/ml-100k.zip

## This unzip method may not work!
# unzip -f ml-100k.zip

### Data Exploration

The lenskit ML100K dataset provides the following: movies, ratings, users

In [3]:
movielens = ML100K('../ml-100k')

In [4]:
ratings = movielens.ratings
ratings.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [5]:
len(ratings)

100000

In [6]:
users = movielens.users
users.head()

,age,gender,occupation,zip
user,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [7]:
len(users)

943

In [8]:
movies = movielens.movies
movies.head()

,title,release,vidrelease,imdb,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [9]:
len(movies)

1682

# Building the Autoencoder


In [31]:
class AutoEncoder():
    
    def build(self, matrix):
                
        encoder_input = Input(shape=(matrix.shape[1]), name='Input', dtype='int8')
        encoder_latent_one = Dense(64, activation='selu', name='Encoder_Latent_One')(encoder_input)
#         encoder_latent_two = Dense(256, activation='selu', name='Encoder_Latent_Two')(encoder_latent_one)
        encoder_latent_three = Dense(32, activation='selu', name='Encoder_Latent_Three')(encoder_latent_one)

        dropout = Dropout(0.35, name='Dropout')(encoder_latent_three)
        
#         decoder_latent_one = Dense(128, activation='selu', name='Decoder_Latent_One')(dropout)
        decoder_latent_two = Dense(64, activation='selu', name='Decoder_Latent_Two')(dropout)
        decoder_output = Dense(matrix.shape[1], activation='softmax', name='Output')(decoder_latent_two)

        model = Model(encoder_input, decoder_output)
        return model
    

In [32]:
uir, _, _= sparse_ratings(ratings, scipy=True)

In [33]:
def rmse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    masked_squared_error = mask_true * K.square((y_true - y_pred))
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
    return K.sqrt(masked_mse)

In [34]:
model = AutoEncoder()
model = model.build(uir)
model.compile(optimizer = Adam(lr=0.001), loss=rmse, metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 1682)]            0         
_________________________________________________________________
Encoder_Latent_One (Dense)   (None, 64)                107712    
_________________________________________________________________
Encoder_Latent_Three (Dense) (None, 32)                2080      
_________________________________________________________________
Dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
Decoder_Latent_Two (Dense)   (None, 64)                2112      
_________________________________________________________________
Output (Dense)               (None, 1682)              109330    
Total params: 221,234
Trainable params: 221,234
Non-trainable params: 0
_____________________________________________________

In [35]:
X = uir.toarray()
y = uir.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [36]:
hist = model.fit(x=X_train, y=y_train,
                  epochs=200,
                  batch_size=32,
                  validation_split=0.2)

Epoch 1/200
22/22 [==============================] - 1s 11ms/step - loss: 3.7402 - accuracy: 0.0073 - val_loss: 3.7385 - val_accuracy: 0.0412
Epoch 2/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7267 - accuracy: 0.0761 - val_loss: 3.7286 - val_accuracy: 0.1059
Epoch 3/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7206 - accuracy: 0.0698 - val_loss: 3.7279 - val_accuracy: 0.1000
Epoch 4/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7193 - accuracy: 0.0667 - val_loss: 3.7271 - val_accuracy: 0.0882
Epoch 5/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7351 - accuracy: 0.0718 - val_loss: 3.7259 - val_accuracy: 0.0941
Epoch 6/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7257 - accuracy: 0.0787 - val_loss: 3.7252 - val_accuracy: 0.0882
Epoch 7/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7234 - accuracy: 0.0898 - val_loss: 3.7243 - val_accuracy: 0.0941
Epoch 8/200


Epoch 59/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7005 - accuracy: 0.0895 - val_loss: 3.7223 - val_accuracy: 0.0882
Epoch 60/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7168 - accuracy: 0.0982 - val_loss: 3.7224 - val_accuracy: 0.1059
Epoch 61/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7159 - accuracy: 0.0823 - val_loss: 3.7224 - val_accuracy: 0.1059
Epoch 62/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7174 - accuracy: 0.0954 - val_loss: 3.7228 - val_accuracy: 0.1118
Epoch 63/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7297 - accuracy: 0.1075 - val_loss: 3.7226 - val_accuracy: 0.1118
Epoch 64/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7037 - accuracy: 0.1056 - val_loss: 3.7223 - val_accuracy: 0.1235
Epoch 65/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7246 - accuracy: 0.0967 - val_loss: 3.7224 - val_accuracy: 0.1118
Epoch 

22/22 [==============================] - 0s 5ms/step - loss: 3.7143 - accuracy: 0.0970 - val_loss: 3.7221 - val_accuracy: 0.0941
Epoch 117/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7270 - accuracy: 0.0895 - val_loss: 3.7222 - val_accuracy: 0.0941
Epoch 118/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7072 - accuracy: 0.1070 - val_loss: 3.7221 - val_accuracy: 0.1000
Epoch 119/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7235 - accuracy: 0.1071 - val_loss: 3.7224 - val_accuracy: 0.0882
Epoch 120/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7111 - accuracy: 0.1041 - val_loss: 3.7225 - val_accuracy: 0.0882
Epoch 121/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7234 - accuracy: 0.0966 - val_loss: 3.7225 - val_accuracy: 0.0941
Epoch 122/200
22/22 [==============================] - 0s 6ms/step - loss: 3.6974 - accuracy: 0.1129 - val_loss: 3.7224 - val_accuracy: 0.1000
Epoch 123/200

22/22 [==============================] - 0s 6ms/step - loss: 3.7179 - accuracy: 0.0973 - val_loss: 3.7225 - val_accuracy: 0.1000
Epoch 174/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7290 - accuracy: 0.0872 - val_loss: 3.7226 - val_accuracy: 0.1000
Epoch 175/200
22/22 [==============================] - 0s 6ms/step - loss: 3.7173 - accuracy: 0.0959 - val_loss: 3.7226 - val_accuracy: 0.0941
Epoch 176/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7053 - accuracy: 0.0905 - val_loss: 3.7227 - val_accuracy: 0.0882
Epoch 177/200
22/22 [==============================] - 0s 6ms/step - loss: 3.6763 - accuracy: 0.1004 - val_loss: 3.7225 - val_accuracy: 0.0882
Epoch 178/200
22/22 [==============================] - 0s 5ms/step - loss: 3.6985 - accuracy: 0.0864 - val_loss: 3.7225 - val_accuracy: 0.0941
Epoch 179/200
22/22 [==============================] - 0s 5ms/step - loss: 3.7062 - accuracy: 0.1074 - val_loss: 3.7225 - val_accuracy: 0.0941
Epoch 180/200

In [37]:
def plot_hist(hist):

    x = np.arange(10)

    fig = go.Figure(data=go.Scatter(y=hist.history['loss']))
    fig.update_layout(title="model loss", 
                    xaxis_title="epoch",
                    yaxis_title="loss",
                    )
    fig.show()

plot_hist(hist)

In [38]:
test_loss = model.evaluate(X_test, X_test)
test_loss

3/3 [==============================] - 0s 2ms/step - loss: 3.7437 - accuracy: 0.0842


[3.743685007095337, 0.08421052992343903]

In [39]:
new_matrix = model.predict(X_train)

In [40]:
print(X_train[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]


In [41]:
print(new_matrix[:5])

[[3.4645340e-12 1.2937381e-19 1.5261524e-20 ... 2.4920434e-19
  3.9274649e-19 2.6312877e-20]
 [5.9108607e-10 2.3786411e-12 1.1261865e-12 ... 1.3120129e-12
  1.5463535e-12 1.2609612e-12]
 [2.8969255e-14 7.2423726e-16 5.5400555e-16 ... 2.9711981e-16
  4.7717474e-16 3.8383332e-16]
 [1.9799973e-12 2.3488965e-17 2.7948156e-17 ... 1.7275401e-17
  8.6469772e-18 3.9192802e-18]
 [1.1503034e-15 4.4682132e-25 1.5302045e-26 ... 5.9651645e-25
  7.6550702e-25 3.1352519e-26]]
